In [28]:
import pandas as pd

file_to_load = "../airbnb/AirBnB 2018/calendar_dec18.csv"
list_to_load = "../airbnb/AirBnB 2018/cleansed_listings_dec18.csv"

cal_2018 = pd.read_csv(file_to_load)
list_2018 = pd.read_csv(list_to_load, low_memory = False)
list_2018_price = list_2018[['id','price']]
list_2018_price = list_2018_price.rename(columns = {'id':'listing_id'})
list_2018_price

,listing_id,price
0,9835,60
1,10803,35
2,12936,159
3,15246,50
4,16760,69
...,...,...
22890,30592332,55
22891,30592345,129
22892,30592362,53
22893,30592590,130


In [29]:
days_cost = pd.merge(cal_2018, list_2018_price, on = 'listing_id', how = 'left')
daily_price_2018 = days_cost.groupby('listing_id').mean()
daily_price_2018.describe()

,price_y
count,22895.000000
mean,148.004368
std,210.883253
min,0.000000
25%,71.000000
50%,111.000000
75%,165.000000
max,12624.000000


In [45]:
#Finding all available booking ID's and total days available
avail_2018 = cal_2018.loc[cal_2018['available'] == 't']
avail_2018 = avail_2018.groupby('listing_id').count()
avail_2018 = pd.merge(avail_2018, list_2018_price, on = 'listing_id', how = 'left')
avail_2018 = avail_2018.rename(columns = {'available':'Number of Days Available',
                                          'price_y': 'Price per Night'})
avail_2018 = avail_2018[['listing_id','Number of Days Available','Price per Night']]
avail_2018

,listing_id,Number of Days Available,Price per Night
0,9835,365,60
1,10803,213,35
2,12936,326,159
3,16760,328,69
4,38271,328,98
...,...,...,...
17392,30592332,82,55
17393,30592345,365,129
17394,30592362,15,53
17395,30592590,349,130


In [49]:
#Finding all booked listing_id's and number of days booked
booked_2018 = cal_2018.loc[cal_2018['available'] == 'f']
booked_2018 = booked_2018.groupby('listing_id').count()
booked_2018 = pd.merge(booked_2018, list_2018_price, on = 'listing_id', how = 'left')
booked_2018 = booked_2018.rename(columns = {'available':'Number of Days Booked',
                                          'price_y': 'Price per Night'})
booked_2018 = booked_2018[['listing_id','Number of Days Booked','Price per Night']]
booked_2018

,listing_id,Number of Days Booked,Price per Night
0,10803,152,35
1,12936,39,159
2,15246,365,50
3,16760,37,69
4,38271,37,98
...,...,...,...
22326,30592128,336,30
22327,30592332,283,55
22328,30592362,350,53
22329,30592590,16,130


In [36]:
cal_group = days_cost.groupby(['listing_id','available']).count()

cal_group

date  price_x  price_y
listing_id available                        
9835       t           365      365      365
10803      f           152        0      152
           t           213      213      213
12936      f            39        0       39
           t           326      326      326
...                    ...      ...      ...
30592362   t            15       15       15
30592590   f            16        0       16
           t           349      349      349
30593623   f           297        0      297
           t            68       68       68

[39728 rows x 3 columns]

In [15]:
cal_2018['date'] = pd.to_datetime(cal_2018['date'])
cal_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8356675 entries, 0 to 8356674
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   listing_id  int64         
 1   date        datetime64[ns]
 2   available   object        
 3   price       object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 255.0+ MB


In [22]:
cal_2018['day'] = cal_2018['date'].dt.day
cal_2018['month'] = cal_2018['date'].dt.month
cal_2018['year'] = cal_2018['date'].dt.year

In [23]:
cal_2018

,listing_id,date,available,price,day,month,year
0,136510,2019-10-15,f,NaN,15,10,2019
1,136510,2019-10-14,f,NaN,14,10,2019
2,136510,2019-10-13,f,NaN,13,10,2019
3,136510,2019-10-12,f,NaN,12,10,2019
4,136510,2019-10-11,f,NaN,11,10,2019
...,...,...,...,...,...,...,...
8356670,30559400,2018-12-11,f,NaN,11,12,2018
8356671,30559400,2018-12-10,f,NaN,10,12,2018
8356672,30559400,2018-12-09,f,NaN,9,12,2018
8356673,30559400,2018-12-08,f,NaN,8,12,2018
